In [1]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime
#for email
import smtplib
from selenium import webdriver
import csv

In [2]:
def get_url(search_term):
    """Generate a url from search term"""
    template="https://www.amazon.com/s?k={}&ref=nb_sb_ss_ts-doa-p_2_8"
    search_term=search_term.replace(' ','+')
    
    #add the term query to url
    url= template.format(search_term)
    #add page queryplaceholder
    url+='&page{}'
    return url


def extract_record(item):
    """extract and return data from single record"""
    
    #description and url
    atag=item.h2.a
    description=atag.text.strip()
    url="https://www.amazon.com"+atag.get("href")
    try:
    # price
        price_parent=item.find("span","a-price")
        price_detail=price_parent.find("span","a-offscreen")
        price=price_detail.text.strip()
    except AttributeError:
        return 
    
    try:
    # review count
        rating=item.i.text.strip()
        count=item.find("span",{"class":"a-size-base s-underline-text"})
        review_count=count.text.strip()
    except AttributeError:
        rating=""
        review_count=""
    
    result=(description,price,rating,review_count,url)
    return result

def main(search_term):
    """Run main program routine"""
    #startup webdriver
   
    
    driver = webdriver.Chrome(executable_path=r'C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe')
    
    records=[]
    url=get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all("div",{"data-component-type":"s-search-result"})

        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    #save data to csv file
    with open('UltraWide_Monitor_Dataset.csv', 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(["Description","price","rating","review count","url"])
        writer.writerows(records)

In [3]:
main('ultrawide monitor')